In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [6]:
dataframe = pd.read_excel("Gem comparison products copy.xlsx")
# dataframe = dataframe.loc[:, ['URL', 'CURRENT PRICE']]
p_index = 0

In [3]:
dataframe.head()

,URL,CURRENT PRICE,QUANTITY,DELIVERY LOCATIONS
0,https://mkp.gem.gov.in/mcb-miniature-circuit-b...,0,10.0,"TAMIL NADU, All India"
1,https://mkp.gem.gov.in/mcb-miniature-circuit-b...,0,NaN,NaN
2,http://mkp.gem.gov.in/mcb-miniature-circuit-br...,0,NaN,NaN
3,https://mkp.gem.gov.in/mcb-miniature-circuit-b...,0,NaN,NaN


In [10]:
type(dataframe["DELIVERY LOCATIONS"][p_index])

str

In [7]:
product_url = dataframe["URL"][p_index]
print(product_url)

https://mkp.gem.gov.in/mcb-miniature-circuit-breakers-c-operation-per-iec/4a-tp-c-mcb/p-5116877-16761642912-cat.html#variant_id=5116877-16761642912


In [47]:
# product_url = "https://mkp.gem.gov.in/electric-ceiling-type-fan-v2-per-374/bajaj-bldc-28-w/p-5116877-29311430059-cat.html"

In [48]:
# product_url = "https://mkp.gem.gov.in/mcb-miniature-circuit-breakers-c-operation-per-iec/20a-sp-c-mcb/p-5116877-295066335-cat.html#variant_id=5116877-295066335"

In [8]:
competitors_url = re.sub(".html.*", "/all_sellers.html", product_url)

In [50]:
competitors_url_response = requests.get(competitors_url)

In [51]:
competitors_url_soup = BeautifulSoup(competitors_url_response.content, 'html.parser')

In [52]:
competitors_prices_table = competitors_url_soup.find('div', attrs={'id': 'sellers-table-wrap'})
competitors_prices_table = competitors_prices_table.find('table')
for div in competitors_prices_table.find_all("div", {'class':'seller-info-table'}):
    div.decompose()

In [53]:
table_rows = competitors_prices_table.find_all('tr')

res = []
for i, tr in enumerate(table_rows):
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=["Sellers", "Offer Price", "Delivery Locations", "Quantity based Discount", "Quantity Available", "Min Quantity / Consignee", "Offer Product As", "Country of Origin"])
df = df[["Offer Price", "Delivery Locations", "Quantity based Discount", "Min Quantity / Consignee"]]

In [54]:
df

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee
0,"₹4,500.00","State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,15
1,"₹2,258.32","State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,10
2,"₹4,600.00","State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...","8 to 100 units = ₹2,341.86",10
3,"₹3,990.00","State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...","8 to 49 units = ₹1,731.66",8
4,"₹4,650.00","State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...","2 to 9 units = ₹785.38\n 10 to 14 units = ₹2,2...",1
5,"₹3,684.89","State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,12
6,"₹4,375.21","State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,2
7,"₹2,410.19",All India,,10
8,"₹2,979.79",All India,,12
9,"₹4,000.00","State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,25


In [65]:
delivery_locations_regex = "|".join(["TAMIL NADU", "All India"])
competitors_details = df[
    df["Delivery Locations"].str.contains(delivery_locations_regex)
]

In [82]:
competitors_details

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee,Minimum Price
0,4500.0,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,15,4500.00
1,2258.32,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,10,2258.32
2,4600.0,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...","8 to 100 units = ₹2,341.86",10,2258.14
3,3990.0,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...","8 to 49 units = ₹1,731.66",8,2258.34
4,4650.0,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...","2 to 9 units = ₹785.38\n 10 to 14 units = ₹2,2...",1,2398.94
5,3684.89,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,12,3684.89
6,4375.21,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,2,4375.21
7,2410.19,All India,,10,2410.19
8,2979.79,All India,,12,2979.79
9,4000.0,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,25,4000.00


In [69]:
competitors_details.loc[:, "Offer Price"] = (
    competitors_details.loc[:, "Offer Price"].str.replace("₹", "").str.replace(",", "").astype(float)
)

In [71]:
def calculate_minimum_price(row, quantity):
    if (
        pd.isnull(row["Quantity based Discount"])
        or row["Quantity based Discount"] == ""
    ):
        return row["Offer Price"]
    else:
        discounts = row["Quantity based Discount"].split("\n")
        final_price = row["Offer Price"]
        for discount in discounts:
            units, discount_amount = discount.split(" = ")
            discount_amount = float(discount_amount.replace("₹", "").replace(",", ""))
            start, end = map(int, re.findall(r"\d+", units))
            if quantity > 0:
                if start <= quantity <= end:
                    return row["Offer Price"] - discount_amount
            else:
                final_price = (
                    row["Offer Price"] - discount_amount
                    if row["Offer Price"] - discount_amount < final_price
                    else final_price
                )
        return final_price

In [81]:
competitors_details.loc[:, "Minimum Price"] = (
    competitors_details.apply(lambda row: calculate_minimum_price(row, 11), axis=1)
)

In [83]:
competitors_details

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee,Minimum Price
0,4500.0,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,15,4500.00
1,2258.32,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,10,2258.32
2,4600.0,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...","8 to 100 units = ₹2,341.86",10,2258.14
3,3990.0,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...","8 to 49 units = ₹1,731.66",8,2258.34
4,4650.0,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...","2 to 9 units = ₹785.38\n 10 to 14 units = ₹2,2...",1,2398.94
5,3684.89,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,12,3684.89
6,4375.21,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,2,4375.21
7,2410.19,All India,,10,2410.19
8,2979.79,All India,,12,2979.79
9,4000.0,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,25,4000.00


In [ ]:
# Drop rows where Min Quantity * Offer Price exceeds 25000
competitors_details = competitors_details[
    competitors_details["Min Quantity / Consignee"].astype(int) * competitors_details["Offer Price"]
    <= 25000
]

In [ ]:
competitors_details

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee,Final Price
0,2099.99,All India,,8,2099.99


In [ ]:
competitors_min_price = competitors_details['Final Price'].min()

In [ ]:
if competitors_min_price == "":
    competitors_min_price = "Put your price"
print(f"Minimum price: {competitors_min_price}")

Minimum price: 2099.99


In [ ]:
competitors_min_price

2099.99

In [ ]:
product_price = dataframe["CURRENT PRICE"][p_index]

In [ ]:
product_price

0

In [ ]:
product_price > competitors_min_price

False